In [42]:
import pandas as pd

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
dataframe = pd.read_csv('news.csv')
dataframe.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [45]:
x = dataframe['text']
y = dataframe['label']

In [46]:
x[100]

"In 2008, in the high-profile Supreme Court gun-rights case called District of Columbia v. Heller, a brief was filed from the eighth floor of the Price Daniel Sr. State Office Building in Austin, Texas, specifically from the corner office of the man who was then the state’s solicitor general, Ted Cruz.\n\nThe brief took a strong stance on the divisive question of whether the Second Amendment establishes an individual right to own guns, or just protects state and local militias. The brief argued forcefully for the first view, writing that “the individual right to keep and bear arms” is a “fundamental right” and that “an individual right that can be altogether abrogated is no right at all.” Thirty attorneys general from other states signed on.\n\nToday, with the White House pushing new gun restrictions and Cruz's candidacy riding on next month's Iowa caucuses, it's no surprise that the Texas Republican would embrace gun rights as a defining issue, holding an event at a firing range and e

In [47]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [49]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [50]:
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [51]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [52]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [54]:
from newspaper import Article
from newsapi.newsapi_client import NewsApiClient

newsapi=NewsApiClient(api_key='e1155f7ec34e4af690db87f96fec058a')

def getNews(url):
    article=Article(url)
    article.download()
    article.parse()
    return {'title':article.title,
            'text':article.text,
            'image':article.top_image}

In [55]:
y[4]

'REAL'

In [56]:
# identify minimum difference 

y_pred = classifier.predict(tfid_x_test)
y_poba = classifier._predict_proba_lr(tfid_x_test)

def difference_value(a,b):
    return a-b if a>b else b-a

min_diff = 1
index = 0
for prob in y_poba:
    diff = difference_value(prob[0], prob[1])
    print(prob, "==>",  diff,y_pred[index],y[index])
    if diff<min_diff:
        print(index ,"MIN : ", min_diff, "=>",y_pred[index],y[index])
        min_diff = diff
    index+=1
print("MIN VALUE ==> ", min_diff)

[0.06816306 0.93183694] ==> 0.8636738731549993 REAL FAKE
0 MIN :  1 => REAL FAKE
[0.73648358 0.26351642] ==> 0.47296716781209064 FAKE FAKE
1 MIN :  0.8636738731549993 => FAKE FAKE
[0.85947705 0.14052295] ==> 0.718954106208898 FAKE REAL
[0.62713091 0.37286909] ==> 0.2542618274758841 FAKE FAKE
3 MIN :  0.47296716781209064 => FAKE FAKE
[0.10037342 0.89962658] ==> 0.7992531657605546 REAL REAL
[0.3281686 0.6718314] ==> 0.34366279075606165 REAL FAKE
[0.49447935 0.50552065] ==> 0.011041290574848484 REAL FAKE
6 MIN :  0.2542618274758841 => REAL FAKE
[0.90043498 0.09956502] ==> 0.8008699553489331 FAKE REAL
[0.1191698 0.8808302] ==> 0.7616603904429975 REAL REAL
[0.76032203 0.23967797] ==> 0.5206440567506094 FAKE REAL
[0.11963801 0.88036199] ==> 0.7607239859464989 REAL REAL
[0.84874486 0.15125514] ==> 0.6974897142674115 FAKE REAL
[0.55784423 0.44215577] ==> 0.11568845666395294 FAKE FAKE
[0.20410429 0.79589571] ==> 0.5917914115245502 REAL FAKE
[0.83645087 0.16354913] ==> 0.6729017403725845 FAKE RE

In [57]:
url = 'https://www.theguardian.com/politics/2022/nov/02/suella-braverman-incendiary-rhetoric-is-truly-shameful'
news_text = getNews(url)['text']

news_pred = classifier.predict(tfvect.transform([news_text]))
news_poba = classifier._predict_proba_lr(tfvect.transform([news_text]))

print(f"prob: {news_poba[0]} || prob_diff : {difference_value(news_poba[0][0],news_poba[0][1])}", " || ", f"pred:{news_pred}")

prob: [0.78026811 0.21973189] || prob_diff : 0.5605362179726963  ||  pred:['FAKE']


In [58]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[568  47]
 [ 41 611]]


In [59]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [60]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['REAL']


In [61]:
fake_news_det("""Go to Article
    "President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.""")

['FAKE']


In [62]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [63]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [64]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [65]:
fake_news_det1("""Go to Article 
    "President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [66]:
fake_news_det1("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday unity march against terrorism.""")

['REAL']


In [67]:
fake_news_det('''U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday unity march against terrorism.''')

['REAL']


In [68]:
fake_news_det("DOHA – Cristiano Ronaldo said he was living a “beautiful moment” after becoming the first player to score in five men’s World Cups, as he helped Portugal beat Ghana 3-2 on Thursday in their Group H opener.He gave reporters a two-minute press conference in which he answered only one question, despite being named Man of the Match.“It’s a beautiful moment in my fifth World Cup,” Ronaldo said.");

['REAL']


In [165]:

from newspaper import Article
from newsapi.newsapi_client import NewsApiClient

newsapi=NewsApiClient(api_key='e24d74bb9cf346e9b533c9e021fc4c2a')

top_headlines= newsapi.get_top_headlines(language='en',page_size=100)


In [ ]:
top_headlines

In [73]:
from random import shuffle
a = [1,2,3,4,5]
b = [6,7,8,9,10]

c = a+b
shuffle(c)
c

[6, 3, 1, 2, 4, 7, 8, 10, 5, 9]

In [131]:
url = 'https://www.theguardian.com/politics/2022/nov/02/suella-braverman-incendiary-rhetoric-is-truly-shameful'
news = getNews(url)


In [75]:
article=Article(url)
article.download()
article.parse()

In [ ]:

news_pred = classifier.predict(tfvect.transform([news_text]))
news_poba = classifier._predict_proba_lr(tfvect.transform([news_text]))

print(f"prob: {news_poba[0]} || prob_diff : {difference_value(news_poba[0][0],news_poba[0][1])}", " || ", f"pred:{news_pred}")

In [81]:

def difference(val1, val2):
    return val1-val2 if val1>val2 else val2-val1

def predict(text):
    text_vector = tfvect.transform([text.replace('\n',"")])
    proba = classifier._predict_proba_lr(text_vector)[0]
    tresh = 0.55
    result = loaded_model.predict(text_vector)
    
    if difference(proba[0], proba[1]) < tresh and proba[0]>0.5:
        return "UNIDENTIFIED"
    else:
        return result[0]

In [109]:
newsClient=NewsApiClient(api_key='e1155f7ec34e4af690db87f96fec058a')



top_headlines= newsClient.get_everything(q="Opinion: A really bad night for some high-profile Trump-backed candidates", language='en')

In [144]:
import urllib.request

uf = urllib.request.urlopen(url)
html = uf.read()